In [1]:
import numpy as np
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
clf = RandomForestClassifier(n_estimators = 100) 
from sklearn import metrics
import pickle
from sklearn import tree
try:
     import flask
except:
    ! conda install --yes flask         

from flask import Flask, request, redirect, url_for, flash, jsonify
import numpy as np
import pickle as p
import json
from pandas import json_normalize

def load (input):
    
    data = pd.read_csv(input, delimiter = ",")
    extraced_ssid_level_pivot =data.pivot(columns=['bssid'], values='level')

    output =pd.concat([data, extraced_ssid_level_pivot] , axis=1)
    if 'level' in output:
        del output['level']
    if 'ssid' in output:
        del output['ssid']
    if 'bssid' in output:
        del output['bssid']
    return output

def get_features_and_target(df , target_to_predicte):
    target=df[target_to_predicte]
    if target_to_predicte in df:
        del df[target_to_predicte]
    features=df
    return features, target


dataset  = load('donnees.txt') ;   # on souhaitera ajouter des donnees dans ce dataset
dataset['salleid'] = dataset['salleid'].astype(str)

dataset = dataset.groupby('positionid').agg("max").fillna(100)
dataset

# 8
X, y =get_features_and_target(dataset, 'salleid')
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33)
clf = RandomForestClassifier(criterion='gini', max_depth=8,n_estimators=50)
clf.fit(X_train, y_train)

y_pred =clf.predict(X_test)
print(clf.score(X_test, y_test))
print(y_pred)

0.5384615384615384
['118' '1' '118' '118' '4' '1' '100' '214' '1' '104' '108' '104' '118'
 '104' '220' '108' '104' '211' '0' '4' '4' '118' '118' '211' '207' '214']


In [2]:
res  = clf.predict_proba(X_test)

In [3]:
def  get_labels():
    labels = []
    for element in y_train:
        if element not in labels:
            labels.append(element)

    labels  = np.array(labels).astype("int")
    labels.sort()
    return labels.tolist()

In [4]:
labels = get_labels()
labels

[0, 1, 4, 100, 104, 108, 113, 118, 207, 211, 214, 220]

In [5]:

# recupere les trois max et leur index   cle valeur 
def get_max(predicted_proba , labels):
    dic = dict()
    for i in range(3):
        print(predicted_proba)
        max_value = max(predicted_proba)
        max_index = predicted_proba.index(max_value)
        print(max_index)
        key = labels[max_index]
        dic[key] = max_value*100
        predicted_proba[max_index]=-1
    return dic

#print(labels)
#print(get_max(res[0].tolist() ,  get_labels()))

In [6]:
pickle.dump(clf, open('modelfinal.pkl','wb')) 

In [7]:
def clean_input(dataset , X_dict): 
    X_dataframe = pd.DataFrame.from_dict(X_dict)
    for feature in X_dataframe: 
        if feature in dataset:
            print(feature)
        else:
            del X_dataframe[feature]
    return  X_dataframe

def threat_input(dataset , X_input):
    X_input= clean_input(dataset , X_input)
    key_array = []
    for feature in X_input:
        key_array.append(feature)
    values = X_input.values
    maj_dataset =  dataset.append(dict(zip(key_array, *values)), ignore_index=True)
    formated_input = maj_dataset.replace(np.nan,100)
    formated_input = formated_input[-1:].values
    return formated_input

In [ ]:
app = Flask(__name__)

@app.route('/predict', methods=['GET','POST'])
def makecalc():
    #print(request.json)
    #print(dataset)
    observation = threat_input(dataset , request.json)
    #print(threat_input(dataset , request.json))
    print ("predicted" , model.predict(observation) , type(model.predict(observation)))
    proba  = model.predict_proba(observation)
    print ("result" , get_max(proba[0].tolist()  , labels))
    return json.dumps(get_max(proba[0].tolist()  , labels))
@app.route('/app', methods=['GET','POST'])
def makecalcc():
    print(request.json)
    return "computed"
if __name__ == '__main__':
    modelfile = 'modelfinal.pkl'
    model = p.load(open(modelfile, 'rb'))
    app.run(debug=False, host='0.0.0.0')
# ici le serveur traite les donnees qu'il recoit 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.28.0.1 - - [06/May/2021 15:29:28] "POST /predict HTTP/1.1" 200 -


predicted ['0'] <class 'numpy.ndarray'>
[0.2351515151515152, 0.10619047619047618, 0.02816017316017316, 0.06967532467532467, 0.05833333333333333, 0.06378787878787878, 0.03857142857142857, 0.008636363636363636, 0.08681818181818182, 0.05285714285714285, 0.22181818181818183, 0.03]
0
[-1, 0.10619047619047618, 0.02816017316017316, 0.06967532467532467, 0.05833333333333333, 0.06378787878787878, 0.03857142857142857, 0.008636363636363636, 0.08681818181818182, 0.05285714285714285, 0.22181818181818183, 0.03]
10
[-1, 0.10619047619047618, 0.02816017316017316, 0.06967532467532467, 0.05833333333333333, 0.06378787878787878, 0.03857142857142857, 0.008636363636363636, 0.08681818181818182, 0.05285714285714285, -1, 0.03]
1
result {0: 23.51515151515152, 214: 22.181818181818183, 1: 10.619047619047619}
[0.2351515151515152, 0.10619047619047618, 0.02816017316017316, 0.06967532467532467, 0.05833333333333333, 0.06378787878787878, 0.03857142857142857, 0.008636363636363636, 0.08681818181818182, 0.05285714285714285,

172.28.0.1 - - [06/May/2021 15:31:33] "POST /predict HTTP/1.1" 200 -


1c:28:af:ce:e7:11
1c:28:af:ce:e7:10
1c:28:af:ce:e7:12
1c:28:af:ce:e7:13
1c:28:af:ce:e7:14
1c:28:af:ce:e7:00
1c:28:af:ce:e7:01
1c:28:af:ce:e7:02
1c:28:af:b2:f7:60
1c:28:af:ce:1b:c2
1c:28:af:b4:1c:e0
1c:28:af:b4:1c:e1
1c:28:af:b4:1c:e2
1c:28:af:ce:1b:c1
1c:28:af:b2:f7:62
1c:28:af:ce:1b:c0
1c:28:af:b2:f7:61
1c:28:af:b2:c8:52
1c:28:af:b2:c8:50
1c:28:af:b2:c8:51
1c:28:af:cd:9f:70
1c:28:af:b2:c8:54
1c:28:af:b2:c8:53
1c:28:af:cd:9f:72
1c:28:af:cd:9f:71
1c:28:af:ce:1b:d0
1c:28:af:ce:1b:d2
1c:28:af:ce:1b:d1
1c:28:af:b2:c8:42
1c:28:af:b2:c8:40
1c:28:af:b2:c8:41
1c:28:af:b4:32:20
1c:28:af:b3:b8:72
1c:28:af:b4:32:22
1c:28:af:cd:23:22
1c:28:af:cd:23:23
1c:28:af:b4:32:21
1c:28:af:cd:23:24
1c:28:af:b3:b8:71
1c:28:af:b3:b8:70
predicted ['4'] <class 'numpy.ndarray'>
[0.08848484848484847, 0.31, 0.09030303030303029, 0.0018181818181818182, 0.0, 0.005454545454545454, 0.02, 0.0036363636363636364, 0.0018181818181818182, 0.01, 0.07848484848484849, 0.39]
11
[0.08848484848484847, 0.31, 0.09030303030303029, 0.00

172.28.0.1 - - [06/May/2021 15:32:46] "POST /predict HTTP/1.1" 200 -


predicted ['0'] <class 'numpy.ndarray'>
[0.2351515151515152, 0.10619047619047618, 0.02816017316017316, 0.06967532467532467, 0.05833333333333333, 0.06378787878787878, 0.03857142857142857, 0.008636363636363636, 0.08681818181818182, 0.05285714285714285, 0.22181818181818183, 0.03]
0
[-1, 0.10619047619047618, 0.02816017316017316, 0.06967532467532467, 0.05833333333333333, 0.06378787878787878, 0.03857142857142857, 0.008636363636363636, 0.08681818181818182, 0.05285714285714285, 0.22181818181818183, 0.03]
10
[-1, 0.10619047619047618, 0.02816017316017316, 0.06967532467532467, 0.05833333333333333, 0.06378787878787878, 0.03857142857142857, 0.008636363636363636, 0.08681818181818182, 0.05285714285714285, -1, 0.03]
1
result {0: 23.51515151515152, 214: 22.181818181818183, 1: 10.619047619047619}
[0.2351515151515152, 0.10619047619047618, 0.02816017316017316, 0.06967532467532467, 0.05833333333333333, 0.06378787878787878, 0.03857142857142857, 0.008636363636363636, 0.08681818181818182, 0.05285714285714285,

In [ ]:
try:
     import pandas_profiling
except:
    ! conda install --yes pandas-profiling   
try:
     import geopandas
except:
    ! conda install --yes geopandas    

In [ ]:
import pandas as pd  
import numpy as np
import pandas_profiling
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
pandas_profiling.ProfileReport(dataset)